# Figures for Paper

## SODA (as same period as SOSE)

In [1]:
import xarray as xr
from matplotlib import pyplot as plt
import gcsfs
import dask
import dask.array as dsa
import numpy as np
import intake
import fsspec
import xmitgcm
from xhistogram.xarray import histogram
from xmitgcm.llcreader.llcmodel import faces_dataset_to_latlon
%matplotlib inline
from fastjmd95 import jmd95numba

In [3]:
wmt_budget_terms = xr.open_zarr(gcsfs.GCSMap('pangeo-tmp/stb2145/SODA/datasets/wmt_budget_terms/wmt_budget_terms.zarr'))
wmt_budget_terms#.load()

<xarray.Dataset>
Dimensions:         (sigma2_bin: 699, time: 1969)
Coordinates:
  * sigma2_bin      (sigma2_bin) float64 1.032e+03 1.032e+03 ... 1.039e+03
  * time            (time) object 1993-01-09 13:00:00 ... 2019-12-19 13:00:00
Data variables:
    dVdt            (time, sigma2_bin) float64 dask.array<chunksize=(493, 175), meta=np.ndarray>
    omega_surf      (time, sigma2_bin) float64 dask.array<chunksize=(493, 175), meta=np.ndarray>
    omega_surf_SLT  (time, sigma2_bin) float64 dask.array<chunksize=(493, 175), meta=np.ndarray>
    omega_surf_TH   (time, sigma2_bin) float64 dask.array<chunksize=(493, 175), meta=np.ndarray>
    omega_sw_TH     (time, sigma2_bin) float64 dask.array<chunksize=(493, 175), meta=np.ndarray>
    omega_total     (time, sigma2_bin) float64 dask.array<chunksize=(493, 175), meta=np.ndarray>

In [4]:
wmt_budget_terms_resample = xr.open_zarr(gcsfs.GCSMap('pangeo-tmp/stb2145/SODA/datasets/wmt_budget_terms/wmt_budget_terms_resample.zarr'))
wmt_budget_terms_resample#.load()

<xarray.Dataset>
Dimensions:         (sigma2_bin: 699, time: 985)
Coordinates:
  * sigma2_bin      (sigma2_bin) float64 1.032e+03 1.032e+03 ... 1.039e+03
  * time            (time) object 1993-01-04 00:00:00 ... 2019-12-14 00:00:00
Data variables:
    dVdt            (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>
    omega_surf      (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>
    omega_surf_SLT  (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>
    omega_surf_TH   (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>
    omega_sw_TH     (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>
    omega_total     (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>
    psi_vel         (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>
    u_binned_left   (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>
    u_binned_right  (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>
    v_binned        (time, sigma2_bin) float64 dask.array<chunksize=(247, 175), meta=np.ndarray>